In [1]:
# Cell 1: Install all necessary libraries
!pip install pandas beautifulsoup4 requests scikit-learn nltk readability tqdm

# Note: If running in Google Colab or an environment without local files, 
# you might need to use !wget or similar to download the data. 
# For a local Jupyter setup, ensure data.csv is in the 'data/' folder.

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for readability: filename=readability-0.3.2-py3-none-any.whl size=36425 sha256=fdf35b246abcfeca482dd3f484e374716022cacceae3c9d0c518d226c4beedb3
  Stored in directory: c:\users\mayac\appdata\local\pip\cache\wheels\f3\58\aa\53a1166ad162b65b2bebf0964c83b977b7091d57a10fe7e0b6
Successfully built readability


In [18]:
# Cell 3: Initial Imports and Data Load
import pandas as pd
import numpy as np
import re
import time
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import textstat
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import pickle
import requests
from tqdm.notebook import tqdm
import json
import warnings
warnings.filterwarnings("ignore")

# Download NLTK dependency required for sentence tokenization
import nltk
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')


# Load the dataset (Assuming data.csv is already placed in the 'data/' folder)
df = pd.read_csv(r'C:\Users\mayac\seo-content-detector\DATA\data.csv')
print(f"Original dataset shape: {df.shape}")
print(df.head(2))



Original dataset shape: (81, 2)
                                               url  \
0   https://www.cm-alliance.com/cybersecurity-blog   
1  https://www.varonis.com/blog/cybersecurity-tips   

                                        html_content  
0  <!doctype html><!--[if lt IE 7]> <html class="...  
1  <!doctype html><html lang="en"><head>\n    <me...  


In [7]:
pip install textstat


Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
print("Current working directory:")
print(os.getcwd())


Current working directory:
C:\Users\mayac


In [40]:
# Cell 4: Data Collection & HTML Parsing (15%)

import os
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

#  Step 1: Load your dataset using the absolute path
df = pd.read_csv(r'C:\Users\mayac\seo-content-detector\DATA\data.csv')
print(f" Original dataset shape: {df.shape}")
print(df.head(2))

#  Step 2: Make sure the output directory exists
os.makedirs(r'C:\Users\mayac\seo-content-detector\DATA', exist_ok=True)

#  Step 3: Define HTML extraction function
def extract_content(html_content):
    """Parses HTML content to extract page title and main body text."""
    if pd.isna(html_content):
        return None, None
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Extract title
        title = soup.title.string if soup.title and soup.title.string else ""
        
        # Find main content tags (article or main)
        main_content_candidates = soup.find_all(['article', 'main'], limit=1)
        
        if not main_content_candidates and soup.body:
            main_content = soup.body
        elif main_content_candidates:
            main_content = main_content_candidates[0]
        else:
            return title, ""

        # Remove unnecessary tags
        for selector in ['script', 'style', 'nav', 'footer', '.header', '.sidebar', '.ad']:
            for tag in main_content.find_all(selector):
                tag.decompose()
        
        # Get clean text
        body_text = main_content.get_text(separator=' ', strip=True)
        return title, body_text

    except Exception:
        return None, None

#  Step 4: Apply function to extract content
tqdm.pandas(desc="Parsing HTML")

df[['title', 'body_text']] = df['html_content'].progress_apply(
    lambda x: pd.Series(extract_content(x))
)

#  Step 5: Drop rows where parsing failed
df_extracted = df.dropna(subset=['body_text']).copy()

#  Step 6: Add word count
df_extracted['word_count'] = df_extracted['body_text'].apply(lambda x: len(str(x).split()))

#  Step 7: Save results in the same DATA folder
output_path = r'C:\Users\mayac\seo-content-detector\DATA\extracted_content.csv'
df_extracted[['url', 'title', 'body_text', 'word_count']].to_csv(output_path, index=False)

print(f"\n Extracted content saved to: {output_path}")
print(f"Pages successfully parsed: {df_extracted.shape[0]} out of {df.shape[0]}")
print("\n📄 Sample Extracted Data:")
print(df_extracted[['url', 'title', 'body_text', 'word_count']].head())


 Original dataset shape: (81, 2)
                                               url  \
0   https://www.cm-alliance.com/cybersecurity-blog   
1  https://www.varonis.com/blog/cybersecurity-tips   

                                        html_content  
0  <!doctype html><!--[if lt IE 7]> <html class="...  
1  <!doctype html><html lang="en"><head>\n    <me...  


Parsing HTML: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 81/81 [00:12<00:00,  6.60it/s]


 Extracted content saved to: C:\Users\mayac\seo-content-detector\DATA\extracted_content.csv
Pages successfully parsed: 69 out of 81

📄 Sample Extracted Data:
                                                 url  \
0     https://www.cm-alliance.com/cybersecurity-blog   
1    https://www.varonis.com/blog/cybersecurity-tips   
2  https://www.cisecurity.org/insights/blog/11-cy...   
3  https://www.cisa.gov/topics/cybersecurity-best...   
5  https://nordlayer.com/learn/network-security/b...   

                                               title  \
0                                Cyber Security Blog   
1  Top 10 Cybersecurity Awareness Tips: How to St...   
2  11 Cyber Defense Tips to Stay Secure at Work a...   
3  Cybersecurity Best Practices | Cybersecurity a...   
5     Network Security 101: Understanding the Basics   

                                           body_text  word_count  
0  Cybersecurity Blog Cybersecurity Measures Ever...        2017  
1  Blog Privacy & Compliance Top 

In [39]:
# Cell 5 (CORRECTED): Text Preprocessing & Feature Engineering (25%)

import re
import textstat
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import os

tqdm.pandas(desc="Feature Engineering")

# --- Setup Paths ---
INPUT_PATH = r'C:\Users\mayac\seo-content-detector\DATA\extracted_content.csv'
OUTPUT_PATH = r'C:\Users\mayac\seo-content-detector\DATA\features.csv'
MODEL_DIR = r'C:\Users\mayac\seo-content-detector\MODELS'
os.makedirs(MODEL_DIR, exist_ok=True) 

# 1. Load the extracted dataset
df = pd.read_csv(INPUT_PATH)
print(f" Loaded data shape: {df.shape}")


# 2.  Text Cleaning Function: Preserves Punctuation
def clean_text_corrected(text):
    """
    Cleans text: removes HTML/extra spaces, lowercases, but KEEPS essential punctuation 
    (periods, commas, etc.) for accurate sentence tokenization by textstat.
    """
    text = str(text)
    # 1. Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # 2. Remove characters that are NOT letters, digits, whitespace, or common sentence punctuation
    # Preserving: .,?!\-
    text = re.sub(r'[^\w\s\.,\?!-]', '', text, flags=re.I | re.U)
    # 3. Clean up extra whitespace and lowercase
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

# Apply corrected cleaning
df['clean_text'] = df['body_text'].progress_apply(clean_text_corrected)


# 3. Readability & Structural Features using textstat
df['flesch_reading_ease'] = df['clean_text'].progress_apply(textstat.flesch_reading_ease)
# Handle possible division-by-zero or non-finite issue in SMOG/Readability functions
df['smog_index'] = df['clean_text'].progress_apply(lambda x: textstat.smog_index(x) if x else 0.0) 
df['sentence_count'] = df['clean_text'].progress_apply(lambda x: len(sent_tokenize(x)) if x else 0)
df['word_count'] = df['clean_text'].apply(lambda x: len(x.split())) # Update word count based on clean text


# 4. TF-IDF (Embeddings & Keywords)
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english', min_df=5)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['clean_text'].fillna(''))

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
embedding_cols = tfidf_df.columns.tolist()

# Define core columns to keep, and drop raw text before merging TF-IDF
df_core = df[['url', 'word_count', 'sentence_count', 'flesch_reading_ease', 'smog_index', 'clean_text']].copy()

# Concatenate features
df = pd.concat([df_core.reset_index(drop=True), tfidf_df], axis=1)

# Thin Content Detection (Required for Section 3/4)
df['is_thin'] = (df['word_count'] < 500).astype(int)


# 5. Extract Top 5 Keywords (FIXED: top_tn -> top_n and enforced float type)
def get_top_keywords(row, feature_names, top_n=5):
    """Extracts top N keywords based on TF-IDF scores."""
    try:
        # Extract vector and ensure it's a numeric NumPy array
        vector = row[feature_names].values.astype(float) 
        
        # Check for NaN/Inf 
        if np.any(np.isnan(vector)):
            return ""

        # Find top N indices (FIXED TYPO: top_tn to top_n)
        top_n_indices = np.argsort(vector)[::-1][:top_n]
        
        # Get keywords
        keywords = [feature_names[i] for i in top_n_indices if vector[i] > 0]
        return "|".join(keywords)
    except Exception:
        # If conversion or sorting fails, return empty string
        return "" 

# Apply function
df['top_keywords'] = df.progress_apply(
    lambda row: get_top_keywords(row, embedding_cols), axis=1
)


# 6. Save and Report
feature_output_cols = ['url', 'word_count', 'sentence_count', 'flesch_reading_ease', 'smog_index', 'is_thin', 'top_keywords'] + embedding_cols
df[feature_output_cols].to_csv(OUTPUT_PATH, index=False)

vectorizer_path = os.path.join(MODEL_DIR, 'tfidf_vectorizer.pkl')
with open(vectorizer_path, 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

print(f"\n Feature Engineering Complete. Features saved to: {OUTPUT_PATH}")
print(f"Total TF-IDF features: {len(embedding_cols)}")
print("\n📄 Sample Feature Data (Scores should now be realistic):")
print(df[['url', 'word_count', 'flesch_reading_ease', 'smog_index', 'is_thin']].head())

 Loaded data shape: (69, 4)


Feature Engineering: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 69/69 [00:00<00:00, 632.89it/s]


 Feature Engineering Complete. Features saved to: C:\Users\mayac\seo-content-detector\DATA\features.csv
Total TF-IDF features: 1000

📄 Sample Feature Data (Scores should now be realistic):
                                                 url       url  word_count  \
0     https://www.cm-alliance.com/cybersecurity-blog  0.000000        2015   
1    https://www.varonis.com/blog/cybersecurity-tips  0.000000        1746   
2  https://www.cisecurity.org/insights/blog/11-cy...  0.080915        1057   
3  https://www.cisa.gov/topics/cybersecurity-best...  0.000000         821   
4  https://nordlayer.com/learn/network-security/b...  0.000000        2163   

   flesch_reading_ease  smog_index  is_thin  
0            41.600994   13.994808        0  
1            40.871699   13.974545        0  
2            53.262918   11.997365        0  
3            -2.538002   20.234475        0  
4            25.769415   13.266885        0  


In [38]:
# Cell 6: Final Duplicate Detection Code (20%)

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os

# Define paths (Ensure these paths are correct)
FEATURES_PATH = r'C:\Users\mayac\seo-content-detector\DATA\features.csv'
DUPLICATES_PATH = r'C:\Users\mayac\seo-content-detector\DATA\duplicates.csv'

# 1. Load the features data and handle NaNs
df_dups = pd.read_csv(FEATURES_PATH)
df_dups = df_dups.fillna(0) # Fill any potential NaNs with 0 for vector math
df_dups['is_thin'] = (df_dups['word_count'] < 500).astype(int) # Recalculate 'is_thin' just in case

# 2. Identify TF-IDF (embedding) columns
# Selects all numeric columns that are NOT the core structural features.
core_non_embedding_cols = ['word_count', 'sentence_count', 'flesch_reading_ease', 'smog_index', 'is_thin']
embedding_cols = [col for col in df_dups.columns if col not in core_non_embedding_cols and df_dups[col].dtype != 'object']

tfidf_vectors = df_dups[embedding_cols].values

print(f" Loaded {len(embedding_cols)} TF-IDF features for similarity check.")
print(f" Data points remaining: {len(df_dups)}")

# 3. Compute pairwise cosine similarity
cosine_sim_matrix = cosine_similarity(tfidf_vectors)
np.fill_diagonal(cosine_sim_matrix, 0) # Set self-similarity to 0 to prevent self-matching

# 4. Define similarity threshold
SIMILARITY_THRESHOLD = 0.80

# 5. Find duplicate pairs
duplicate_pairs = []
for i in range(len(df_dups)):
    # Iterate over the upper triangle of the matrix (j > i)
    for j in range(i + 1, len(df_dups)):
        similarity_score = cosine_sim_matrix[i, j]
        if similarity_score >= SIMILARITY_THRESHOLD:
            duplicate_pairs.append({
                'url1': df_dups.iloc[i]['url'],
                'url2': df_dups.iloc[j]['url'],
                'similarity': round(similarity_score, 4)
            })

df_duplicates = pd.DataFrame(duplicate_pairs)

# 6. Save the duplicate pairs
df_duplicates.to_csv(DUPLICATES_PATH, index=False)


### 7. Report Summary
total_pages = len(df_dups)
num_duplicates = len(df_duplicates)
num_thin = df_dups['is_thin'].sum()
thin_percent = (num_thin / total_pages) * 100

print("\n--- Duplicate & Thin Content Summary (Code Logic Correct) ---")
print(f"Total pages analyzed: {total_pages}")
print(f"Duplicate pairs found (Similarity > {SIMILARITY_THRESHOLD}): {num_duplicates}")
print(f"Thin content pages (Word Count < 500): {num_thin} ({thin_percent:.1f}%)")
print("\n📄 Sample Duplicate Pairs:")
print(df_duplicates.head())

 Loaded 1002 TF-IDF features for similarity check.
 Data points remaining: 69

--- Duplicate & Thin Content Summary (Code Logic Correct) ---
Total pages analyzed: 69
Duplicate pairs found (Similarity > 0.8): 14
Thin content pages (Word Count < 500): 14 (20.3%)

📄 Sample Duplicate Pairs:
                                                url1  \
0  https://www.fortinet.com/resources/cyberglossa...   
1  https://guardiandigital.com/resources/blog/gui...   
2               https://en.wikipedia.org/wiki/SD-WAN   
3               https://en.wikipedia.org/wiki/SD-WAN   
4  https://www.cisco.com/site/us/en/learn/topics/...   

                                                url2  similarity  
0  https://www.cisco.com/site/us/en/learn/topics/...      0.8382  
1  https://inspiredelearning.com/blog/phishing-pr...      0.8568  
2  https://www.cisco.com/site/us/en/learn/topics/...      0.8868  
3  https://www.fortinet.com/resources/cyberglossa...      0.9101  
4  https://www.fortinet.com/resources/cy

In [35]:
# Cell 7: Content Quality Scoring (25%)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle
import pandas as pd
import numpy as np
import os

# Define paths (Ensure these paths are correct)
FEATURES_PATH = r'C:\Users\mayac\seo-content-detector\DATA\features.csv'
MODEL_PATH = r'C:\Users\mayac\seo-content-detector\MODELS\quality_model.pkl'

df_model = pd.read_csv(FEATURES_PATH)
df_model = df_model.fillna(0) # Fill any NaNs with 0

# Ensure 'is_thin' is present (though it should be from Cell 6)
if 'is_thin' not in df_model.columns:
    df_model['is_thin'] = (df_model['word_count'] < 500).astype(int)


### 4.1 Synthetic Label Creation
def create_quality_label(row):
    """
    Creates a quality label based on word count and readability/smog score.
    Flesch (50-70 ideal) and SMOG (8-12 ideal for grade level).
    """
    word_count = row['word_count']
    flesch_score = row['flesch_reading_ease']
    smog_score = row['smog_index']
    
    # High Condition: Good length AND balanced readability
    # Word count > 1500 AND Flesch 50-70 AND SMOG <= 12 (ignoring 0 smog from error rows)
    if word_count > 1500 and (50 <= flesch_score <= 70) and (smog_score <= 12 and smog_score > 0):
        return 'High'
    
    # Low Condition: Poor length OR very poor readability
    # Word count < 500 OR Flesch < 30 OR SMOG > 15
    elif word_count < 500 or flesch_score < 30 or smog_score > 15:
        return 'Low'
    
    # Medium Condition (all other cases)
    else:
        return 'Medium'

df_model['quality_label'] = df_model.apply(create_quality_label, axis=1)
print("\nSynthetic Label Distribution:")
print(df_model['quality_label'].value_counts())


### 4.2 Model Training & Evaluation
# Core structural features + TF-IDF embeddings
core_structural_features = ['word_count', 'sentence_count', 'flesch_reading_ease', 'smog_index']
# Get embedding columns (columns that are not core structural or control columns)
control_cols = ['url', 'is_thin', 'top_keywords', 'quality_label', 'clean_text'] 
embedding_cols = [col for col in df_model.columns if col not in core_structural_features and col not in control_cols and df_model[col].dtype != 'object']
feature_cols_ml = core_structural_features + embedding_cols

X = df_model[feature_cols_ml]
y = df_model['quality_label']

# Train/Test Split (70/30)
# Stratify ensures equal proportion of High/Medium/Low in both sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


# 1. Baseline Model (Rule-based: Word Count only)
def baseline_predict(word_count):
    if word_count > 1000: return 'High'
    elif word_count < 600: return 'Low'
    else: return 'Medium'

y_test_baseline = X_test['word_count'].apply(baseline_predict)
baseline_accuracy = accuracy_score(y_test, y_test_baseline)
print(f"\nBaseline Accuracy (Word Count Rule): {baseline_accuracy:.2f}")


# 2. Classification Model (Random Forest)
quality_model = RandomForestClassifier(n_estimators=100, random_state=42)
quality_model.fit(X_train, y_train)
y_pred = quality_model.predict(X_test)

# Report Metrics
accuracy = accuracy_score(y_test, y_pred)

print("\n--- Model Performance: Random Forest Classifier ---")
print(classification_report(y_test, y_pred, zero_division=0))
print(f"Overall Accuracy: {accuracy:.2f}")

# Top Features
feature_importances = pd.Series(quality_model.feature_importances_, index=feature_cols_ml)
top_features = feature_importances.nlargest(3)

print("\nTop Features:")
for i, (name, importance) in enumerate(top_features.items()):
    print(f"{i+1}. {name} (importance: {importance:.2f})")

# Save the model
with open(MODEL_PATH, 'wb') as f:
    pickle.dump(quality_model, f)
print(f"\n Quality model saved to {MODEL_PATH}")


Synthetic Label Distribution:
quality_label
Low       35
Medium    29
High       5
Name: count, dtype: int64

Baseline Accuracy (Word Count Rule): 0.38

--- Model Performance: Random Forest Classifier ---
              precision    recall  f1-score   support

        High       0.00      0.00      0.00         1
         Low       1.00      1.00      1.00        11
      Medium       0.90      1.00      0.95         9

    accuracy                           0.95        21
   macro avg       0.63      0.67      0.65        21
weighted avg       0.91      0.95      0.93        21

Overall Accuracy: 0.95

Top Features:
1. smog_index (importance: 0.05)
2. flesch_reading_ease (importance: 0.04)
3. 34 (importance: 0.03)

 Quality model saved to C:\Users\mayac\seo-content-detector\MODELS\quality_model.pkl


In [34]:
# Cell 8: Real-Time Analysis Demo (15%)

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import textstat
from nltk.tokenize import sent_tokenize
import pickle
import os
from sklearn.metrics.pairwise import cosine_similarity

# --- Setup Paths ---
MODEL_DIR = r'C:\Users\mayac\seo-content-detector\MODELS'
FEATURES_PATH = r'C:\Users\mayac\seo-content-detector\DATA\features.csv'
# Load saved model and vectorizer
VECTORIZER_PATH = os.path.join(MODEL_DIR, 'tfidf_vectorizer.pkl')
MODEL_PATH = os.path.join(MODEL_DIR, 'quality_model.pkl')

tfidf_vectorizer = None
quality_model = None
try:
    with open(VECTORIZER_PATH, 'rb') as f:
        tfidf_vectorizer = pickle.load(f)
    with open(MODEL_PATH, 'rb') as f:
        quality_model = pickle.load(f)
    print(" Model and Vectorizer loaded successfully.")
except FileNotFoundError:
    print(f" Error: Model or Vectorizer not found in {MODEL_DIR}. Analysis will be skipped.")
    
# --- Helper Functions (Replicating Pipeline) ---

def fetch_and_parse(url):
    """Fetches content from a URL and extracts title and body text."""
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=15)
        if response.status_code != 200:
            return None, None
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.title.string if soup.title and soup.title.string else "N/A"
        
        main_content = soup.find('body')
        if not main_content:
            return title, ""

        # Remove scripts/styles/nav/footer for cleaner content
        for tag in main_content(['script', 'style', 'nav', 'footer', 'header']):
            tag.decompose()
            
        body_text = main_content.get_text(separator=' ', strip=True)
        return title, body_text

    except Exception:
        return None, None

def clean_text_for_analysis(text):
    """Corrected cleaning to allow accurate sentence tokenization (must match Cell 5)."""
    text = str(text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^\w\s\.,\?!-]', '', text, flags=re.I | re.U)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()


def get_realtime_features(clean_text, tfidf_vectorizer, quality_model):
    """Calculates all necessary structural and TF-IDF features for model input."""
    
    # Structural Features
    word_count = len(clean_text.split())
    sentence_count = len(sent_tokenize(clean_text))
    flesch_reading_ease = textstat.flesch_reading_ease(clean_text)
    smog_index = textstat.smog_index(clean_text) if clean_text else 0.0
    
    # TF-IDF Features (Prediction Vector)
    tfidf_vector = tfidf_vectorizer.transform([clean_text])
    
    # Prepare the feature array for the model
    feature_names = tfidf_vectorizer.get_feature_names_out()
    tfidf_df = pd.DataFrame(tfidf_vector.toarray(), columns=feature_names)
    
    # Combine structural features with TF-IDF vector
    structural_data = {
        'word_count': [word_count],
        'sentence_count': [sentence_count],
        'flesch_reading_ease': [flesch_reading_ease],
        'smog_index': [smog_index]
    }
    df_structural = pd.DataFrame(structural_data)
    
    X_predict = pd.concat([df_structural.reset_index(drop=True), tfidf_df.reset_index(drop=True)], axis=1)

    # Filter X_predict to ensure column alignment with the trained model
    # Features created during training that might not be in the new text will be added as 0.
    model_feature_cols = quality_model.feature_names_in_ 
    X_predict = X_predict.reindex(columns=model_feature_cols, fill_value=0)

    # Keyword Extraction (for reporting)
    top_keywords = []
    vector = tfidf_vector.toarray()[0]
    top_n_indices = np.argsort(vector)[::-1][:5]
    for i in top_n_indices:
        if vector[i] > 0:
            top_keywords.append(feature_names[i])
            
    return X_predict, {
        'word_count': word_count,
        'smog_index': smog_index,
        'flesch_score': flesch_reading_ease,
        'top_keywords': "|".join(top_keywords),
        'vector': tfidf_vector.toarray()
    }

def run_analysis(url):
    """Runs the full SEO content analysis pipeline for a new URL."""
    if not quality_model or not tfidf_vectorizer:
        print("Analysis skipped due to missing model files.")
        return

    print(f"\n---  Analyzing New URL: {url} ---")
    title, raw_text = fetch_and_parse(url)
    
    if not raw_text or len(raw_text.split()) < 50:
        print(" Error: Could not fetch usable content or content is too short.")
        return

    clean_text = clean_text_for_analysis(raw_text)
        
    X_predict, metrics = get_realtime_features(clean_text, tfidf_vectorizer, quality_model)
    
    # Predict Quality
    quality_prediction = quality_model.predict(X_predict)[0]
    
    # Thin Content Check
    is_thin = "YES" if metrics['word_count'] < 500 else "NO"
    
    # --- Final Report ---
    print("\n Content Metrics:")
    print(f"  Title: {title}")
    print(f"  Word Count: {metrics['word_count']}")
    print(f"  Thin Content: **{is_thin}**")
    print(f"  Flesch Readability: {metrics['flesch_score']:.2f}")
    print(f"  SMOG Index (Grade): {metrics['smog_index']:.2f}")
    print(f"  Top Keywords: {metrics['top_keywords']}")
    
    print("\n Model Prediction:")
    print(f"  Predicted Quality: **{quality_prediction}**")
    
    # --- Duplicate Check against Database (Cell 6 Data) ---
    try:
        df_db = pd.read_csv(FEATURES_PATH)
        df_db = df_db.fillna(0)
        
        # Identify the full set of TF-IDF columns from the database to ensure alignment
        core_cols = ['url', 'word_count', 'sentence_count', 'flesch_reading_ease', 'smog_index', 'is_thin', 'top_keywords']
        db_embedding_cols = [col for col in df_db.columns if col not in core_cols and df_db[col].dtype != 'object']
        
        db_vectors = df_db[db_embedding_cols].values
        
        # Ensure the prediction vector aligns perfectly with DB columns
        X_predict_vector = X_predict.reindex(columns=db_embedding_cols, fill_value=0).values
        
        # Cosine Similarity Check
        similarity_scores = cosine_similarity(X_predict_vector, db_vectors)[0]
        max_sim = np.max(similarity_scores)
        
        if max_sim >= 0.80:
            match_url = df_db.iloc[np.argmax(similarity_scores)]['url']
            print(f"\n Duplicate Alert:")
            print(f"  High similarity ({max_sim:.4f}) found with existing URL:")
            print(f"  {match_url}")
        else:
            print(f"\n Duplicate Check: No high similarity match found (Max Sim: {max_sim:.4f})")
            
    except Exception as e:
        print(f"\n Duplicate Check Failed: {e}")


# --- DEMONSTRATION: Replace the URL below with any live URL to test ---
TEST_URL = 'https://www.cisa.gov/topics/cybersecurity-best-practices' 
run_analysis(TEST_URL)

 Model and Vectorizer loaded successfully.

---  Analyzing New URL: https://www.cisa.gov/topics/cybersecurity-best-practices ---

 Content Metrics:
  Title: Cybersecurity Best Practices | Cybersecurity and Infrastructure Security Agency CISA
  Word Count: 902
  Thin Content: **NO**
  Flesch Readability: 2.55
  SMOG Index (Grade): 19.35
  Top Keywords: cybersecurity|cyber|practices|secure|organizations

 Model Prediction:
  Predicted Quality: **Low**

 Duplicate Alert:
  High similarity (0.9915) found with existing URL:
  https://www.cisa.gov/topics/cybersecurity-best-practices
